In [1]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv("C:/Users/ckaus/Desktop/Machine Learning/Project/spam.csv",encoding="ISO-8859-1")

In [8]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


Data Cleaning

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [10]:
#dropping the last 3 columns
df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)

In [11]:
df.sample(5)

,v1,v2
1014,ham,"Ya, told..she was asking wats matter?"
358,ham,"I'm an actor. When i work, i work in the eveni..."
3261,ham,O shore are you takin the bus
4840,ham,"Hmmm.... Mayb can try e shoppin area one, but ..."
755,ham,So why didnt you holla?


In [12]:
df.rename(columns={'v1':'target','v2':'text'},inplace=True)
df.sample(5)

,target,text
4309,spam,Someone U know has asked our dating service 2 ...
672,spam,Get ur 1st RINGTONE FREE NOW! Reply to this ms...
3618,spam,8007 25p 4 Alfie Moon's Children in Need song ...
1458,ham,"Fighting with the world is easy, u either win ..."
4756,ham,"hey, looks like I was wrong and one of the kap..."


In [13]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [14]:
df['target'] = encoder.fit_transform(df['target'])

In [15]:
df.head()

,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [16]:
df.isnull().sum()

target    0
text      0
dtype: int64

In [17]:
df.duplicated().sum()

403

In [18]:
df = df.drop_duplicates(keep='first')

In [19]:
df.duplicated().sum()

0

In [20]:
df.shape

(5169, 2)

In [21]:
df['target'].value_counts()

0    4516
1     653
Name: target, dtype: int64

In [22]:
df_copy = df['text'].copy()

<!-- !pip install nltk
import nltk
nltk.download('stopwords') -->

In [23]:
import string
from nltk.corpus import stopwords
def text_preprocess(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    return " ".join(text)

In [29]:
df_copy = df_copy.apply(text_preprocess)

In [31]:
df_copy.sample(5)

360                       Ha ha cool cool chikku chikkuDB
3432    Christmas occasion Celebrated Reflection UR Va...
3524                 free today haf 2 pick parents tonite
525          Hi wont b ard 4 christmas enjoy n merry xmas
988     Yun ahthe ubi one say Ì wan call tomorrowcall ...
Name: text, dtype: object

Training the model

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [33]:
vectorizer = TfidfVectorizer()

In [34]:
df_mat = vectorizer.fit_transform(df_copy)
df_mat

<5169x9376 sparse matrix of type '<class 'numpy.float64'>'
	with 43208 stored elements in Compressed Sparse Row format>

In [35]:
message_train, message_test, spam_nospam_train, spam_nospam_test = train_test_split(df_mat, 
                                                        df['target'], test_size=0.3, random_state=20)

Building the model

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [37]:
lb_model = LogisticRegression(solver='liblinear', penalty='l1')
lb_model.fit(message_train, spam_nospam_train)
pred = lb_model.predict(message_test)
accuracy_score(spam_nospam_test,pred)

0.941972920696325

In [38]:
def stemmer (text):
    text = text.split()
    words = ""
    for i in text:
            stemmer = SnowballStemmer("english")
            words += (stemmer.stem(i))+" "
    return words

In [39]:
from nltk.stem import SnowballStemmer
df_copy = df_copy.apply(stemmer)
vectorizer = TfidfVectorizer()
df_mat = vectorizer.fit_transform(df_copy)

In [40]:
message_train, message_test, spam_nospam_train, spam_nospam_test = train_test_split(df_mat, 
                                                        df['target'], test_size=0.3, random_state=20)

In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lb_model = LogisticRegression(solver='liblinear', penalty='l1')
lb_model.fit(message_train, spam_nospam_train)
pred = lb_model.predict(message_test)
accuracy_score(spam_nospam_test,pred)

0.9477756286266924